In [ ]:
from models.resnet50_backbone import ResNet50Backbone
from src.data_loader import ShanghaiTechDataset
import torch
from torch.utils.data import DataLoader
from torchvision import transforms


In [ ]:

model = ResNet50Backbone()
model.load_state_dict(torch.load("../models/resnet50_finetuned.pth"))
model.eval()

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

In [ ]:


val_dataset = ShanghaiTechDataset("../data/ShanghaiTech", part="part_A", split="test_data", transform=transform)
val_loader = DataLoader(val_dataset, batch_size=1)

mae = mse = 0
with torch.no_grad():
    for img, count_map in val_loader:
        count = count_map.sum(dim=(1,2)).unsqueeze(1).float()
        output = model(img)
        mae += torch.abs(output - count).item()
        mse += ((output - count)**2).item()

print("MAE:", mae / len(val_dataset))
print("RMSE:", (mse / len(val_dataset))**0.5)